In [ ]:
" @author: Pedro :') "
""
#Importamos módulos y librerías necesarias
import itertools
import matplotlib.pylab as plt
import matplotlib.cm as cm
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Flatten
import tensorflow.keras.layers as layers
from tensorflow.keras import Model
from tensorflow import keras

import PIL
import os
from PIL import Image
import glob

from tensorflow import math
import numpy as np
import matplotlib.cm as cm
from PIL import Image
from tensorflow.math import confusion_matrix
import sys

In [ ]:
## DEFINIMOS FUNCIONES IMPORTANTES PARA GRADCAM

#Función de convertir la imagen en un determinado path a una array
def get_img_array(img_path, img_size):
    img = keras.preprocessing.image.load_img(img_path, target_size=img_size)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    img_array = np.expand_dims(array, axis=0)
    return img_array
        
#Función de crear la parte "operacional" de GradCam
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names,last_conv_layer):
    
    # Modelo que mapea la imagen de entrada a la última capa convolucional dónde se calculará la activación
    conv_model = keras.Model(model.inputs, last_conv_layer.output)
    
    p=0
    predictions = []

    # Modelo que mapea las activaciones a la salida final
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)
        
    with tf.GradientTape() as tape:
        
        # Calcula activacion del modelo base convolucional
        last_conv_layer_output = conv_model(img_array)
        tape.watch(last_conv_layer_output)
        
        # Calcula la predicción con modelo clasificador, para la clase más probable
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class = int(np.array(top_pred_index))
        p = p + top_class
        top_class_channel = preds[:, top_pred_index]
        predictions.append(top_class_channel)

    # Obtenemos el gradiente en la capa final clasificadora con respecto a la salida del modelo base convolucional
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # Vector de pesos: medias del gradiente por capas,
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # salida de la última capa convolucional
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    
    # saliencia es la respuesta promedio de la última capa convolucional
    saliency = np.mean(last_conv_layer_output, axis=-1)
    saliency = np.maximum(saliency, 0) / np.max(saliency)
    
    # Multiplicación de cada canal por el vector de pesos
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
        
    # Heatmap: promedio de cada canal por su peso
    grad_cam = np.mean(last_conv_layer_output, axis=-1)
    grad_cam = np.maximum(grad_cam, 0) / np.max(grad_cam)
    return grad_cam,saliency,predictions,top_class

#Función de representar la imagen en una ventana emergente y guardarla en la carpeta elegida
def show_hotmap (img_path, heatmap, title='Heatmap', alpha=0.6, cmap='jet', axisOnOff='off'):
    cam_path = r'C:\Users\pop1md\Desktop\NO_segmentacion\Pruebaaa\1.jpg'
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    superimposed_img.save(cam_path)
    Image.open(cam_path).show()

In [ ]:
BATCH_SIZE = 40

imgOK_path = r'/content/drive/MyDrive/TFG/Fotos latas/OK'
imgNOK_path = r'/content/drive/MyDrive/TFG/Fotos latas/NOK'
dirListingOK = os.listdir(imgOK_path)
dirListingNOK = os.listdir(imgNOK_path)
nOK = len(dirListingOK)
nNOK = len(dirListingNOK)

#Definimos el dataset ya definido entre imágenes OK y NOK
data_dir = r'/content/drive/MyDrive/TFG/Fotos latas'
def build_dataset(subset, validation_split=0.3):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      labels="inferred",
      label_mode="categorical",
      subset=subset,
      image_size=(300,300),
      shuffle=True,
      seed=100,
      validation_split=validation_split)

#Especificamos que este Dataset de imágenes son las que utilizaremos para el entrenamiento
train_ds = build_dataset(subset="training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)

normalization_layer = tf.keras.layers.Layer()
preprocessing_model = tf.keras.Sequential([normalization_layer])
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

#Especificamos que este otro Dataset son las que utilizaremos para la validación
val_ds = build_dataset(subset="validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

#Seleccionamos el modelo que vamos a utilizar y representamos el summary
modelvgg16 = VGG16(include_top=False,
              input_shape=(300,300,3),
              weights=None,
              classes=2)

#Modificamos las últimas capas de salida para sólo obtener 2 posibles outputs
x = modelvgg16.output
x = layers.Flatten()(x)
x = layers.Dense(units=4096, activation='relu')(x)
x = layers.Dense(units=4096, activation='relu')(x)
x = layers.Dense(units=2, activation='sigmoid')(x)

model = Model(modelvgg16.input, x)

model.summary()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
#Ahora viene la parte de reproducir el GradCam, para ello asignamos nombres a diferentes capas
last_layer_output = model.layers[-1].output
last_conv_layer = model.layers[-6]
last_conv_layer_name = 'block5_conv3'
classifier_layer_names =  ['block5_pool',
                            'flatten',
                            'dense',
                            'dense_1',
                            'dense_2']


#Cargamos las imágenes sobre las que vamos a aplicar el GradCam seleccionando todo el fichero
img_size = (300,300)
folder_path_o = r'/content/drive/MyDrive/TFG/Fotos latas/OK/*.jpg'
folder_path_n = r'/content/drive/MyDrive/TFG/Fotos latas/NOK/*.jpg'
fileo_with_path_list = glob.glob(folder_path_o)         #glob.glob nos permites obtener el nombre
filen_with_path_list = glob.glob(folder_path_n)         #de la imagen junto con el path completo

#Entrenamos y validamos el modelo (parte más larga)
hist = model.fit_generator(
    train_ds,
    epochs=80,
    verbose = 1,
    validation_data = val_ds).history

#Recorremos cada una de las imágenes para ver su MAPA DE CALOR GRADCAM

n_numbero = 0
r_preds=[]
t_class = []

import matplotlib.image as mpimg

for fileo in fileo_with_path_list:
    n_numbero = n_numbero + 1
    imgo = Image.open(fileo)
    img_path = fileo

    img_array = get_img_array(img_path,img_size)
    grad_cam, saliency, predictions, top_class = make_gradcam_heatmap(img_array,
                                                  model,
                                                  last_layer_output,
                                                  classifier_layer_names,
                                                  last_conv_layer)
    
    show_hotmap(img_path, heatmap=grad_cam, title=f'Grad Cam: {model.name}')
    show_hotmap(img_path, heatmap=saliency, title=f'Saliencia: {model.name}')

    r_preds.append(predictions)
    t_class.append(top_class)
    
n_numbern = 100000
for filen in filen_with_path_list:
    n_numbern = n_numbern + 1
    imgn = Image.open(filen)
    img_path = filen

    img_array = get_img_array(img_path,img_size)
    grad_cam, saliency, predictions, top_class = make_gradcam_heatmap(img_array,
                                                  model,
                                                  last_layer_output,
                                                  classifier_layer_names,
                                                  last_conv_layer)
    
    show_hotmap(img_path, heatmap=grad_cam, title=f'Grad Cam: {model.name}')
    show_hotmap(img_path, heatmap=saliency, title=f'Saliencia: {model.name}')

    r_preds.append(predictions)
    t_class.append(top_class)

labels = np.hstack((np.ones(nOK), np.zeros(nNOK)))
print(t_class)
print(labels)

conf_matrix = tf.math.confusion_matrix(labels=labels,
                                        predictions=t_class,
                                        num_classes=2,
                                        weights=None)
print(conf_matrix)

In [ ]:
# Creamos una lista dónde aparezca la foto, y si es un FP, FN, VP o VN

l_labels=[]
for elem in labels:
  l_labels.append(elem)
indices_NOK=[]
V_P=0
V_N=0
F_P=0
F_N=0
for elem in range(len(l_labels)):
  if elem<len(fileo_with_path_list):
    image_name=fileo_with_path_list[elem]
  else:
    image_name=filen_with_path_list[elem-len(fileo_with_path_list)]

  dif= l_labels[elem]-t_class[elem]
  if dif==0 and l_labels[elem]==1:
    V_P=V_P+1
  elif dif==0 and l_labels[elem]==0:
    V_N=V_N+1
  elif dif==1:
    image_name = fileo_with_path_list[elem]
    indices_NOK.append([elem, image_name, 'Falso Negativo'])
    F_P=F_P+1
  elif dif==-1:
    indices_NOK.append([elem, image_name, 'Falso Positivo'])
    F_N=F_N+1
print([V_P, F_P, F_N, V_N])
print(indices_NOK)


In [ ]:
# Representamos el mapa de saliencia y el descenso de gradiente para comprobar que se fija en la parte correcta

import matplotlib.image as mpimg

for filen in indices_NOK:
    n_numbern = n_numbern + 1
    imgn = Image.open(filen[1])
    img_path = filen[1]

    img_array = get_img_array(img_path,img_size)
    grad_cam, saliency, predictions, top_class = make_gradcam_heatmap(img_array,
                                                  model,
                                                  last_layer_output,
                                                  classifier_layer_names,
                                                  last_conv_layer)
    show_hotmap(img_path, heatmap=grad_cam, title=f'Grad Cam: {model.name}')
    show_hotmap(img_path, heatmap=saliency, title=f'Saliencia: {model.name}')
    print(filen)
    plt.subplot(121)
    plt.title('GradCam')
    plt.imshow(grad_cam, 'jet')
    plt.subplot(122)
    plt.imshow(saliency, 'jet')
    plt.title('Saliencia')
    plt.show()
    img = mpimg.imread(filen[1])
    imgplot = plt.imshow(img)
    plt.show()